In [31]:
import pandas as pd
import pymysql
import os
from bs4 import BeautifulSoup
import requests
from sqlalchemy import create_engine

In [2]:
url = "http://vegetablesfruitsgrains.com/list-of-vegetables/"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'html.parser')
results = soup.find_all('div', id="container")

vegetables = []

for result in results:
    # Error handling
    try:
        # Identify and return title of listing
        title = result.find_all('li')
        
        for veg in title:
            vegetables.append(veg.text)
            
    except AttributeError as e:
        print(e)

In [107]:
#manually manipulate list of vegetables for those not included
vegetables.append("Potatoes")

In [3]:
products = pd.read_csv(r"C:\Users\cloca\Desktop\DataAnalytics\etl_project\instacart_market_basket_data\products.csv")
products.head()

,product_id,product_name,aisle_id,department_id
0,1,Chocolate Sandwich Cookies,61,19
1,2,All-Seasons Salt,104,13
2,3,Robust Golden Unsweetened Oolong Tea,94,7
3,4,Smart Ones Classic Favorites Mini Rigatoni Wit...,38,1
4,5,Green Chile Anytime Sauce,5,13


In [4]:
dept = pd.read_csv(r"C:\Users\cloca\Desktop\DataAnalytics\etl_project\instacart_market_basket_data\departments.csv")
dept.head()

,department_id,department
0,1,frozen
1,2,other
2,3,bakery
3,4,produce
4,5,alcohol


In [5]:
isproduce = dept['department']=='produce'
dept_produce = dept[isproduce]

In [14]:
produce_merge = pd.merge(products,
                dept_produce,
                on='department_id')
produce_merge.head(1684)

product_names = produce_merge['product_name']
produce_merge.head()

,product_id,product_name,aisle_id,department_id,department
0,31,White Pearl Onions,123,4,produce
1,43,Organic Clementines,123,4,produce
2,45,European Cucumber,83,4,produce
3,66,European Style Spring Mix,123,4,produce
4,89,Yogurt Fruit Dip Sliced Apples,123,4,produce


In [110]:
produce_merge["is_vegetable"] = False

for index, row in produce_merge.iterrows():
    words = row["product_name"].split()
    
    for word in words:
        if word in vegetables:
            produce_merge.loc[index, "is_vegetable"]=True
produce_merge.head()

,product_id,product_name,aisle_id,department_id,department,is_vegetable
2,45,European Cucumber,83,4,produce,True
5,99,Local Living Butter Lettuce,83,4,produce,True
6,120,Cauliflower Florettes,123,4,produce,True
7,142,Arugula Salad,123,4,produce,True
16,347,"Vegetable Salad Kit, Sweet Kale",123,4,produce,True


In [109]:
produce_veg = produce_merge[produce_merge.is_vegetable != False]
print(produce_merge)

      product_id                                       product_name  aisle_id  \
2             45                                  European Cucumber        83   
5             99                        Local Living Butter Lettuce        83   
6            120                              Cauliflower Florettes       123   
7            142                                      Arugula Salad       123   
16           347                    Vegetable Salad Kit, Sweet Kale       123   
20           391                      Organic Yukon Gold Potato Bag        83   
22           494                               Yellow Cherry Tomato        83   
24           581                          Garnet Sweet Potato (Yam)        83   
26           740                                       Celery Heart       123   
27           748                                            Romaine       123   
29           793              Organic Spinach Quinoa Powermeal Bowl       123   
30           857            

In [9]:
#For some reason my orig file was converted from UTF-8 to ANSI, so I converted and renamed
order_products__prior = pd.read_csv(r"C:\Users\cloca\Desktop\DataAnalytics\etl_project\instacart_market_basket_data\order_products__prior2.txt")
order_products__prior.head()

,order_id,product_id,add_to_cart_order,reordered
0,2,33120,1,1
1,2,28985,2,1
2,2,9327,3,0
3,2,45918,4,1
4,2,30035,5,0


In [19]:
produce_merger = pd.merge(produce_veg,
                order_products__prior,
                on='product_id')

In [124]:
#All vegetables
veggies = produce_merger['product_name'].value_counts()
veggies = pd.DataFrame(veggies)
veggies.reset_index(inplace=True)
veggies.head()

,index,product_name
0,Organic Baby Spinach,241921
1,Organic Hass Avocado,213584
2,Organic Avocado,176815
3,Organic Yellow Onion,113426
4,Organic Garlic,109778


In [92]:
top50veggies = produce_merger['product_name'].value_counts().nlargest(50)
top50veggies = pd.DataFrame(top50veggies)
top50veggies.reset_index(inplace=True)
top50veggies.columns={"vegetables", "order count"}

,vegetables,order count
0,Organic Baby Spinach,241921
1,Organic Hass Avocado,213584
2,Organic Avocado,176815
3,Organic Yellow Onion,113426
4,Organic Garlic,109778


In [116]:
connection_string = "root:locandr091@localhost/etl_project"
engine = create_engine(f'mysql://{connection_string}')

In [117]:
top50veggies.to_sql(name='top50veggies', con=engine, if_exists='append', index=True)

In [118]:
engine.table_names()

['top50veggies']